## Intro to deep learning for medical imaging 

[MD.ai](https://www.md.ai)

## Lesson 3. RSNA Pneumonia Detection Challenge (Kaggel API)

The [Radiological Society of North America](http://www.rsna.org/) Pneumonia Detection Challenge: https://www.kaggle.com/c/rsna-pneumonia-detection-challenge

This notebook covers the basics of parsing the competition dataset, training using a detector basd on the [Mask-RCNN algorithm](https://arxiv.org/abs/1703.06870) for object detection and instance segmentation.  

This notebook is developed by [MD.ai](https://www.md.ai), the platform for medical AI. 
This notebook requires Google's [TensorFlow](https://www.tensorflow.org/) machine learning framework.

**Intro to deep learning for medical imaging lessons**

- Lesson 1. Classification of chest vs. adominal X-rays using TensorFlow/Keras [Github](https://github.com/mdai/ml-lessons/blob/master/lesson1-xray-images-classification.ipynb) [Annotator](https://public.md.ai/annotator/project/PVq9raBJ)

- Lesson 2. Lung X-Rays Semantic Segmentation using UNets. [Github](https://github.com/mdai/ml-lessons/blob/master/lesson2-lung-xrays-segmentation.ipynb)
[Annotator](https://public.md.ai/annotator/project/aGq4k6NW/workspace) 

- Lesson 3. RSNA Pneumonia detection using Kaggle data format [Github](https://github.com/mdai/ml-lessons/blob/master/lesson3-rsna-pneumonia-detection-kaggle.ipynb) [Annotator](https://public.md.ai/annotator/project/LxR6zdR2/workspace) 
  
- Lesson 3. RSNA Pneumonia detection using MD.ai python client library [Github](https://github.com/mdai/ml-lessons/blob/master/lesson3-rsna-pneumonia-detection-mdai-client-lib.ipynb) [Annotator](https://public.md.ai/annotator/project/LxR6zdR2/workspace) 

In [1]:
import os 
import sys

import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob 
import scipy.signal as signal
#import skimage.morphology as sm

# enter your Kaggle credentionals here
os.environ['KAGGLE_USERNAME']="lixiaoqi"
os.environ['KAGGLE_KEY']="a96b56107f574e1cf4e14e3822ca2c93"

os.chdir('E:\学习\ml-lessons')
# Root directory of the project
ROOT_DIR = os.path.abspath('./lesson3-data')

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, 'logs')

#if not os.path.exists(ROOT_DIR):
#    os.makedirs(ROOT_DIR)
os.chdir(ROOT_DIR)####

# Import Mask RCNN
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

train_dicom_dir = os.path.join(ROOT_DIR, 'stage_1_train_images')
test_dicom_dir = os.path.join(ROOT_DIR, 'stage_1_test_images')

'''
- dicom_fps is a list of the dicom image path and filenames 
- image_annotions is a dictionary of the annotations keyed by the filenames
- parsing the dataset returns a list of the image filenames and the annotations dictionary
'''
# 将图片的绝对路径以list方式保存起来，这样就可以通过索引的方式直接获取图像路径及名称
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

#image_fps保存了list类型的图像路径及名称，image_annotations保存了每一张图像所对应的识别框的信息
def parse_dataset(dicom_dir, anns): 
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations 

# set color for class
def get_colors_for_class_ids(class_ids):
    colors = []
    for class_id in class_ids:
        if class_id == 1:
            colors.append((.941, .204, .204))
    return colors

#  直方图均衡
def histep(img, nbr_bins=256):
    """Histogram equalization of a grayscale image."""
    # 获取直方图p(r)
    imhist, bins = np.histogram(img.flatten(), nbr_bins, normed=True)
    
    # 获取T(r)
    cdf = imhist.cumsum()  # cumulative distribution function
    cdf = 255 * cdf / cdf[-1]
    
    #获取s, 并用s替换原始图像对应的灰度值
    result = np.interp(img.flatten(), bins[:-1], cdf)
    return result.reshape(img.shape), cdf

# The following parameters have been selected to reduce running time for demonstration purposes 
# These are not optimal 
# 为在RSNA pneumonia数据集上训练模型而重写mask_rcnn中的Config类
class DetectorConfig(Config):
    # 给该configuration取一个可识别的名字
    NAME = 'pneumonia'
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8#4,8
    #BATCH_SIZE是上面两个值的乘积
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 2   # background + 1 pneumonia classes
    
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 256  ####64,256,512
    IMAGE_MAX_DIM = 256
    
    RPN_ANCHOR_SCALES = (16,32, 64, 128)#(16, 32, 64, 128)   #(32, 64, 128, 256)小一号
    
    TRAIN_ROIS_PER_IMAGE = 32  #16,32
    
    MAX_GT_INSTANCES = 4
    
    DETECTION_MAX_INSTANCES = 3
    DETECTION_MIN_CONFIDENCE = 0.78   #待定
    DETECTION_NMS_THRESHOLD = 0.01
    
    # RPN_TRAIN_ANCHORS_PER_IMAGE = 16
    STEPS_PER_EPOCH = 200  #100
    # TOP_DOWN_PYRAMID_SIZE = 32
    
config = DetectorConfig()
#config.display()

class DetectorDataset(utils.Dataset):
    """Dataset class for training pneumonia detection on the RSNA pneumonia dataset.
    """

    def __init__(self, image_fps, image_annotations, orig_height, orig_width):
        super().__init__(self)
        
        # 添加类
        self.add_class('pneumonia', 1, 'Lung Opacity')
   
        # 添加图像
        for i, fp in enumerate(image_fps):
            annotations = image_annotations[fp]
            self.add_image('pneumonia', image_id=i, path=fp, 
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)
            
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        ds = pydicom.read_file(fp)
        image = ds.pixel_array
        
        # image = signal.medfilt(image, kernel_size=5)   # 对图片进行中值滤波
        # image = image.astype(np.uint8)
        # image = cv2.equalizeHist(image)  # 对图片进行直方图均衡处理
        #image = sm.closing(image,sm.disk(5))  #用边长为5的圆形滤波器进行闭运算滤波
        
        image, _ = histep(image)   # 对图片进行直方图均衡处理
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            class_ids = np.zeros((count,), dtype=np.int32)
            for i, a in enumerate(annotations):
                if a['Target'] == 1:
                    x = int(a['x'])
                    y = int(a['y'])
                    w = int(a['width'])
                    h = int(a['height'])
                    mask_instance = mask[:, :, i].copy()
                    cv2.rectangle(mask_instance, (x, y), (x+w, y+h), 255, -1)
                    mask[:, :, i] = mask_instance
                    class_ids[i] = 1
        return mask.astype(np.bool), class_ids.astype(np.int32)

    
# 训练集
f=open(os.path.join(ROOT_DIR, 'stage_1_train_labels.csv'))
anns = pd.read_csv(f)
image_fps, image_annotations = parse_dataset(train_dicom_dir, anns=anns)
# Original DICOM image size: 1024 x 1024
ORIG_SIZE = 1024

# Modify this line to use more or fewer images for training/validation. 
# To use all images, do: image_fps_list = list(image_fps)
image_fps_list = list(image_fps)#[:1000] 
# 将数据分为训练集和验证集
# split ratio is set to 0.9 vs. 0.1 (train vs. validation, respectively)
#sorted(image_fps_list)
random.seed(42)
random.shuffle(image_fps_list)

# 训练集和验证集
val_size = 1500
image_fps_val = image_fps_list[:val_size]
image_fps_train = image_fps_list[val_size:]
'''
validation_split = 0.1
split_index = int((1 - validation_split) * len(image_fps_list))
image_fps_train = image_fps_list[:split_index]
image_fps_val = image_fps_list[split_index:]
'''
print('训练和测试样例个数:',len(image_fps_train), len(image_fps_val))

# prepare the training dataset
dataset_train = DetectorDataset(image_fps_train, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_train.prepare()

# Show annotation(s) for a DICOM image 
# choice() 方法返回一个列表，元组或字符串的随机项。
# test_fp = random.choice(image_fps_train)
# print(image_annotations[test_fp])

# prepare the validation dataset
dataset_val = DetectorDataset(image_fps_val, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_val.prepare()

model = modellib.MaskRCNN(mode='training', config=config, model_dir=MODEL_DIR)

# Image augmentation (light but constant)
augmentation = iaa.Sequential([
        #iaa.Fliplr(0.5),
        iaa.OneOf([  # geometric transform
            iaa.Affine(
                scale={"x": (0.98, 1.02), "y": (0.98, 1.04)},
                translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
                rotate=(-2, 2),
                shear=(-1, 1),
            ),
            iaa.PiecewiseAffine(scale=(0.001, 0.025)),
        ]),
        iaa.OneOf([  # brightness or contrast
            iaa.Multiply((0.9, 1.1)),
            iaa.ContrastNormalization((0.9, 1.1)),
        ]),
        iaa.OneOf([  # blur or sharpen
            iaa.GaussianBlur(sigma=(0.0, 0.1)),
            iaa.Sharpen(alpha=(0.0, 0.1)),
        ]),
])

# 加载预训练的COCO权重
COCO_MODEL_PATH = "./Mask_RCNN/samples/coco/mask_rcnn_coco.h5"

#设定权重的初始化方式，有imagenet，coco,last三种
init_with = "coco"
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    model.load_weights(model.find_last()[1], by_name=True)


#model_path = 'E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181007T1430\mask_rcnn_pneumonia_0016.h5'
#model.load_weights(model_path, by_name=True)

NUM_EPOCHS = 30
LEARNING_RATE = 0.004
# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")

# 头两个epoch使用较大的learning rate，并且只训练heads部分
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE*2, 
            epochs=2, 
            layers='heads',
            augmentation=None)  # no need to augment yet
history = model.keras_model.history.history

model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE, 
            epochs=6, 
            layers='all',
            augmentation=augmentation)
new_history = model.keras_model.history.history
for k in new_history: history[k] = history[k] + new_history[k]

# 后面的epoch使用更小的learning rate
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/5, 
            epochs=16, 
            layers='all',
            augmentation=augmentation)
new_history = model.keras_model.history.history
for k in new_history: history[k] = history[k] + new_history[k]
            
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/50, 
            epochs=35,
            layers='all',
            augmentation=augmentation)

'''model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/500, 
            epochs=NUM_EPOCHS, 
            layers='all',
            augmentation=augmentation)'''
print('train complete')

Using TensorFlow backend.


训练和测试样例个数: 24184 1500

Starting at epoch 0. LR=0.008

Checkpoint Path: E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181010T2307\mask_rcnn_pneumonia_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv

 53/200 [======>.......................] - ETA: 49:53 - loss: 5.4967 - rpn_class_loss: 0.0464 - rpn_bbox_loss: 2.1422 - mrcnn_class_loss: 1.4589 - mrcnn_bbox_loss: 0.7702 - mrcnn_mask_loss: 1.07 - ETA: 29:34 - loss: 4.8156 - rpn_class_loss: 0.0446 - rpn_bbox_loss: 2.0425 - mrcnn_class_loss: 1.0108 - mrcnn_bbox_loss: 0.7032 - mrcnn_mask_loss: 1.01 - ETA: 23:01 - loss: 4.3568 - rpn_class_loss: 0.0389 - rpn_bbox_loss: 1.6150 - mrcnn_class_loss: 1.0291 - mrcnn_bbox_loss: 0.7469 - mrcnn_mask_loss: 0.92 - ETA: 19:02 - loss: 3.8169 - rpn_class_loss: 0.0355 - rpn_bbox_loss: 1.3561 - mrcnn_class_loss: 0.8537 - mrcnn_bbox_loss: 0.7275 - mrcnn_mask_loss: 0.84 - ETA: 17:49 - loss: 3.4746 - rpn_class_loss: 0.0328 - rpn_bbox_loss: 1.1985 - mrcnn_class_loss: 0.7463 - mrcnn_bbox_loss: 0.7182 - mrcnn_mask_loss: 0.77 - ETA: 16:25 - loss: 3.2828 - rpn_class_loss: 0.0305 - rpn_bbox_loss: 1.0756 - mrcnn_class_loss: 0.7090 - mrcnn_bbox_loss: 0.7341 - mrcnn_mask_loss: 0.73 - ETA: 15:58 - loss: 3.0712 - rpn_c

106/200 [==============>...............] - ETA: 8:05 - loss: 1.9850 - rpn_class_loss: 0.0239 - rpn_bbox_loss: 0.4874 - mrcnn_class_loss: 0.3684 - mrcnn_bbox_loss: 0.6457 - mrcnn_mask_loss: 0.459 - ETA: 8:02 - loss: 1.9770 - rpn_class_loss: 0.0237 - rpn_bbox_loss: 0.4858 - mrcnn_class_loss: 0.3652 - mrcnn_bbox_loss: 0.6440 - mrcnn_mask_loss: 0.458 - ETA: 7:57 - loss: 1.9681 - rpn_class_loss: 0.0236 - rpn_bbox_loss: 0.4848 - mrcnn_class_loss: 0.3622 - mrcnn_bbox_loss: 0.6412 - mrcnn_mask_loss: 0.456 - ETA: 7:54 - loss: 1.9609 - rpn_class_loss: 0.0235 - rpn_bbox_loss: 0.4832 - mrcnn_class_loss: 0.3605 - mrcnn_bbox_loss: 0.6389 - mrcnn_mask_loss: 0.454 - ETA: 7:48 - loss: 1.9529 - rpn_class_loss: 0.0234 - rpn_bbox_loss: 0.4819 - mrcnn_class_loss: 0.3573 - mrcnn_bbox_loss: 0.6371 - mrcnn_mask_loss: 0.453 - ETA: 7:43 - loss: 1.9491 - rpn_class_loss: 0.0233 - rpn_bbox_loss: 0.4821 - mrcnn_class_loss: 0.3561 - mrcnn_bbox_loss: 0.6352 - mrcnn_mask_loss: 0.452 - ETA: 7:40 - loss: 1.9410 - rpn_cl

159/200 [======================>.......] - ETA: 5:03 - loss: 1.7878 - rpn_class_loss: 0.0211 - rpn_bbox_loss: 0.4493 - mrcnn_class_loss: 0.3047 - mrcnn_bbox_loss: 0.5868 - mrcnn_mask_loss: 0.426 - ETA: 5:01 - loss: 1.7835 - rpn_class_loss: 0.0210 - rpn_bbox_loss: 0.4485 - mrcnn_class_loss: 0.3038 - mrcnn_bbox_loss: 0.5851 - mrcnn_mask_loss: 0.425 - ETA: 5:00 - loss: 1.7832 - rpn_class_loss: 0.0209 - rpn_bbox_loss: 0.4491 - mrcnn_class_loss: 0.3035 - mrcnn_bbox_loss: 0.5848 - mrcnn_mask_loss: 0.424 - ETA: 4:56 - loss: 1.7791 - rpn_class_loss: 0.0209 - rpn_bbox_loss: 0.4471 - mrcnn_class_loss: 0.3033 - mrcnn_bbox_loss: 0.5835 - mrcnn_mask_loss: 0.424 - ETA: 4:52 - loss: 1.7761 - rpn_class_loss: 0.0208 - rpn_bbox_loss: 0.4464 - mrcnn_class_loss: 0.3025 - mrcnn_bbox_loss: 0.5825 - mrcnn_mask_loss: 0.423 - ETA: 4:49 - loss: 1.7756 - rpn_class_loss: 0.0207 - rpn_bbox_loss: 0.4463 - mrcnn_class_loss: 0.3025 - mrcnn_bbox_loss: 0.5823 - mrcnn_mask_loss: 0.423 - ETA: 4:46 - loss: 1.7738 - rpn_cl

...

200/200 [==============================] - ETA: 3:18 - loss: 1.0571 - rpn_class_loss: 0.0055 - rpn_bbox_loss: 0.2432 - mrcnn_class_loss: 0.1534 - mrcnn_bbox_loss: 0.3029 - mrcnn_mask_loss: 0.352 - ETA: 3:12 - loss: 1.0576 - rpn_class_loss: 0.0055 - rpn_bbox_loss: 0.2427 - mrcnn_class_loss: 0.1539 - mrcnn_bbox_loss: 0.3032 - mrcnn_mask_loss: 0.352 - ETA: 3:07 - loss: 1.0569 - rpn_class_loss: 0.0055 - rpn_bbox_loss: 0.2426 - mrcnn_class_loss: 0.1539 - mrcnn_bbox_loss: 0.3030 - mrcnn_mask_loss: 0.351 - ETA: 3:03 - loss: 1.0557 - rpn_class_loss: 0.0055 - rpn_bbox_loss: 0.2421 - mrcnn_class_loss: 0.1535 - mrcnn_bbox_loss: 0.3028 - mrcnn_mask_loss: 0.351 - ETA: 2:59 - loss: 1.0566 - rpn_class_loss: 0.0055 - rpn_bbox_loss: 0.2423 - mrcnn_class_loss: 0.1542 - mrcnn_bbox_loss: 0.3027 - mrcnn_mask_loss: 0.351 - ETA: 2:53 - loss: 1.0557 - rpn_class_loss: 0.0055 - rpn_bbox_loss: 0.2422 - mrcnn_class_loss: 0.1541 - mrcnn_bbox_loss: 0.3024 - mrcnn_mask_loss: 0.351 - ETA: 2:48 - loss: 1.0561 - rpn_cl

 53/200 [======>.......................] - ETA: 21:19 - loss: 1.2024 - rpn_class_loss: 0.0076 - rpn_bbox_loss: 0.2465 - mrcnn_class_loss: 0.1793 - mrcnn_bbox_loss: 0.3753 - mrcnn_mask_loss: 0.39 - ETA: 20:28 - loss: 1.1706 - rpn_class_loss: 0.0079 - rpn_bbox_loss: 0.2596 - mrcnn_class_loss: 0.1723 - mrcnn_bbox_loss: 0.3438 - mrcnn_mask_loss: 0.38 - ETA: 17:25 - loss: 1.1937 - rpn_class_loss: 0.0077 - rpn_bbox_loss: 0.3350 - mrcnn_class_loss: 0.1478 - mrcnn_bbox_loss: 0.3306 - mrcnn_mask_loss: 0.37 - ETA: 16:58 - loss: 1.1325 - rpn_class_loss: 0.0066 - rpn_bbox_loss: 0.2880 - mrcnn_class_loss: 0.1528 - mrcnn_bbox_loss: 0.3265 - mrcnn_mask_loss: 0.35 - ETA: 17:19 - loss: 1.1766 - rpn_class_loss: 0.0068 - rpn_bbox_loss: 0.3029 - mrcnn_class_loss: 0.1688 - mrcnn_bbox_loss: 0.3309 - mrcnn_mask_loss: 0.36 - ETA: 18:44 - loss: 1.2000 - rpn_class_loss: 0.0071 - rpn_bbox_loss: 0.2988 - mrcnn_class_loss: 0.1728 - mrcnn_bbox_loss: 0.3452 - mrcnn_mask_loss: 0.37 - ETA: 17:48 - loss: 1.2446 - rpn_c

106/200 [==============>...............] - ETA: 12:13 - loss: 1.1059 - rpn_class_loss: 0.0062 - rpn_bbox_loss: 0.2529 - mrcnn_class_loss: 0.1598 - mrcnn_bbox_loss: 0.3227 - mrcnn_mask_loss: 0.36 - ETA: 12:05 - loss: 1.0985 - rpn_class_loss: 0.0062 - rpn_bbox_loss: 0.2506 - mrcnn_class_loss: 0.1588 - mrcnn_bbox_loss: 0.3203 - mrcnn_mask_loss: 0.36 - ETA: 12:06 - loss: 1.0996 - rpn_class_loss: 0.0061 - rpn_bbox_loss: 0.2505 - mrcnn_class_loss: 0.1595 - mrcnn_bbox_loss: 0.3207 - mrcnn_mask_loss: 0.36 - ETA: 12:02 - loss: 1.1045 - rpn_class_loss: 0.0062 - rpn_bbox_loss: 0.2561 - mrcnn_class_loss: 0.1586 - mrcnn_bbox_loss: 0.3209 - mrcnn_mask_loss: 0.36 - ETA: 11:57 - loss: 1.1028 - rpn_class_loss: 0.0062 - rpn_bbox_loss: 0.2547 - mrcnn_class_loss: 0.1591 - mrcnn_bbox_loss: 0.3205 - mrcnn_mask_loss: 0.36 - ETA: 11:48 - loss: 1.1068 - rpn_class_loss: 0.0061 - rpn_bbox_loss: 0.2563 - mrcnn_class_loss: 0.1586 - mrcnn_bbox_loss: 0.3226 - mrcnn_mask_loss: 0.36 - ETA: 11:42 - loss: 1.1054 - rpn_c

159/200 [======================>.......] - ETA: 7:48 - loss: 1.0931 - rpn_class_loss: 0.0059 - rpn_bbox_loss: 0.2583 - mrcnn_class_loss: 0.1514 - mrcnn_bbox_loss: 0.3169 - mrcnn_mask_loss: 0.360 - ETA: 7:43 - loss: 1.0931 - rpn_class_loss: 0.0059 - rpn_bbox_loss: 0.2577 - mrcnn_class_loss: 0.1514 - mrcnn_bbox_loss: 0.3174 - mrcnn_mask_loss: 0.360 - ETA: 7:39 - loss: 1.0932 - rpn_class_loss: 0.0059 - rpn_bbox_loss: 0.2577 - mrcnn_class_loss: 0.1516 - mrcnn_bbox_loss: 0.3174 - mrcnn_mask_loss: 0.360 - ETA: 7:33 - loss: 1.0931 - rpn_class_loss: 0.0058 - rpn_bbox_loss: 0.2578 - mrcnn_class_loss: 0.1514 - mrcnn_bbox_loss: 0.3174 - mrcnn_mask_loss: 0.360 - ETA: 7:27 - loss: 1.0969 - rpn_class_loss: 0.0059 - rpn_bbox_loss: 0.2610 - mrcnn_class_loss: 0.1509 - mrcnn_bbox_loss: 0.3179 - mrcnn_mask_loss: 0.361 - ETA: 7:22 - loss: 1.0957 - rpn_class_loss: 0.0059 - rpn_bbox_loss: 0.2603 - mrcnn_class_loss: 0.1507 - mrcnn_bbox_loss: 0.3178 - mrcnn_mask_loss: 0.361 - ETA: 7:15 - loss: 1.0954 - rpn_cl

200/200 [==============================] - ETA: 3:20 - loss: 1.0858 - rpn_class_loss: 0.0057 - rpn_bbox_loss: 0.2533 - mrcnn_class_loss: 0.1507 - mrcnn_bbox_loss: 0.3163 - mrcnn_mask_loss: 0.359 - ETA: 3:15 - loss: 1.0872 - rpn_class_loss: 0.0057 - rpn_bbox_loss: 0.2540 - mrcnn_class_loss: 0.1511 - mrcnn_bbox_loss: 0.3164 - mrcnn_mask_loss: 0.360 - ETA: 3:10 - loss: 1.0862 - rpn_class_loss: 0.0057 - rpn_bbox_loss: 0.2536 - mrcnn_class_loss: 0.1513 - mrcnn_bbox_loss: 0.3160 - mrcnn_mask_loss: 0.359 - ETA: 3:05 - loss: 1.0854 - rpn_class_loss: 0.0057 - rpn_bbox_loss: 0.2529 - mrcnn_class_loss: 0.1513 - mrcnn_bbox_loss: 0.3158 - mrcnn_mask_loss: 0.359 - ETA: 3:00 - loss: 1.0852 - rpn_class_loss: 0.0057 - rpn_bbox_loss: 0.2527 - mrcnn_class_loss: 0.1513 - mrcnn_bbox_loss: 0.3159 - mrcnn_mask_loss: 0.359 - ETA: 2:55 - loss: 1.0854 - rpn_class_loss: 0.0057 - rpn_bbox_loss: 0.2526 - mrcnn_class_loss: 0.1515 - mrcnn_bbox_loss: 0.3160 - mrcnn_mask_loss: 0.359 - ETA: 2:50 - loss: 1.0844 - rpn_cl

In [2]:
# select trained model 
dir_names = next(os.walk(model.model_dir))[1]
key = config.NAME.lower()
dir_names = filter(lambda f: f.startswith(key), dir_names)
dir_names = sorted(dir_names)

if not dir_names:
    import errno
    raise FileNotFoundError(
            errno.ENOENT,
            "Could not find model directory under {}".format(model.model_dir))          
    
fps = []
# Pick last directory
for d in dir_names: 
    dir_name = os.path.join(model.model_dir, d)
    # Find the last checkpoint
    checkpoints = next(os.walk(dir_name))[2]
    checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
    checkpoints = sorted(checkpoints)
    if not checkpoints:
        print('No weight files in {}'.format(dir_name))
    else: 
      
        checkpoint = os.path.join(dir_name, checkpoints[-1])
        fps.append(checkpoint)

model_path = sorted(fps)[-1]
print('Found model {}'.format(model_path))

Found model E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181010T2307\mask_rcnn_pneumonia_0035.h5


In [8]:
model_path = 'E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181010T2307\mask_rcnn_pneumonia_0013.h5'

class InferenceConfig(DetectorConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

# Get filenames of test dataset DICOM images
test_image_fps = get_dicom_fps(test_dicom_dir)

# Make predictions on test images, write out sample submission 
def predict(image_fps, filepath='submission.csv', min_conf=0.95): 
    
    # assume square image
    resize_factor = ORIG_SIZE / config.IMAGE_SHAPE[0]
    with open(filepath, 'w') as file:
      file.write("patientId,PredictionString\n")
      
      for image_id in tqdm(image_fps): 
        ds = pydicom.read_file(image_id)
        image = ds.pixel_array
        # image = signal.medfilt(image, kernel_size=5)   # 对图片进行中值滤波
        # image = image.astype(np.uint8)
        # image = cv2.equalizeHist(image)  # 对图片进行直方图均衡处理
        image, _ = histep(image)   # 对图片进行直方图均衡处理
          
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1) 
        image, window, scale, padding, crop = utils.resize_image(
            image,
            min_dim=config.IMAGE_MIN_DIM,
            min_scale=config.IMAGE_MIN_SCALE,
            max_dim=config.IMAGE_MAX_DIM,
            mode=config.IMAGE_RESIZE_MODE)
            
        patient_id = os.path.splitext(os.path.basename(image_id))[0]

        results = model.detect([image])
        r = results[0]

        out_str = ""
        out_str += patient_id 
        out_str += ","
        assert( len(r['rois']) == len(r['class_ids']) == len(r['scores']) )
        if len(r['rois']) == 0: 
            pass
        else: 
            num_instances = len(r['rois'])
            for i in range(num_instances): 
                if r['scores'][i] > min_conf: 
                    out_str += ' '
                    out_str += str(round(r['scores'][i], 2))
                    out_str += ' '

                    # x1, y1, width, height 
                    x1 = r['rois'][i][1]
                    y1 = r['rois'][i][0]
                    width = r['rois'][i][3] - x1 
                    height = r['rois'][i][2] - y1 
                    bboxes_str = "{} {} {} {}".format(x1*resize_factor, y1*resize_factor, \
                                                      width*resize_factor, height*resize_factor)    
                    out_str += bboxes_str

        file.write(out_str+"\n")

sample_submission_fp = 'Mask_RCNN/submission.csv'
predict(test_image_fps, filepath=sample_submission_fp)

Loading weights from  E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181010T2307\mask_rcnn_pneumonia_0013.h5
Re-starting from epoch 13


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [08:45<00:00,  1.88it/s]
